In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#### Loading the Necessary Elements

In [ ]:
df_clean = pd.DataFrame('./elements/df_clean.csv', parse_dates=['date'])


In [ ]:
num_pipe = joblib.load('./elements/num_pipe.joblib')

In [ ]:
data_dic = joblib.load('./elements/procssd.joblib')

In [ ]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [ ]:
model = tf.keras.models.load_model('./model/model.h5')

In [ ]:
train_pred = model.predict(data_dic['train_seq'])

In [ ]:
test_pred = model.predict(data_dic['test_seq'])

In [ ]:
train_i_pred= num_pipe.inverse_transform(np.repeat(train_pred, 3, axis=1))

### Testing the Model Performance

In [ ]:
train_rmse = np.sqrt(mean_squared_error(data_dic['train_label'], train_i_pred[:, -1].reshape(-1, 1)))

In [ ]:
test_i_pred= num_pipe.inverse_transform(np.repeat(test_pred, 3, axis=1))

In [ ]:
test_rmse = np.sqrt(mean_squared_error(data_dic['test_label'], test_i_pred[:, -1].reshape(-1, 1)))

In [ ]:
rmse_vals = pd.DataFrame({'Train RMSE': train_rmse, 'Test RMSE': test_rmse}, index=['RMSE']).T.rename(columns={0:'RMSE'})

In [ ]:
plt.figure(figsize=(4, 12))
sns.set_theme(style="whitegrid")
ax = sns.barplot(data=rmse_vals, x=rmse_vals.index, y='RMSE')
plt.title('RMSE for Test and Train Data')
plt.ylim(top=15)
for p in ax.patches:
    # get the height of each bar
    height = p.get_height()
    # adding text to each bar
    ax.text(x=p.get_x()+(p.get_width()/2),  # x-coordinate position of data label, padded to be in the middle of the bar
            y=height+0.2,  # y-coordinate position of data label, padded 100 above bar
            # data label, formatted to have 4 decimals
            s='{: .2f}'.format(height),
            ha='center')  # sets horizontal alignment (ha) to center
plt.savefig('./rmse.png', dpi=300, format=None, metadata=None,
            bbox_inches=None, pad_inches=0.1
            )
plt.show()


### Predicting Into the Future(July, August, September)

In [ ]:
# 3months * 1550 = 4650
# Predicting 3 months into the future
jul_sep = model.predict(data_dic['test_seq'][-4650:])

In [ ]:
jul_sep = num_pipe.inverse_transform(np.repeat(jul_sep, 3, axis=1))[
    :, -1].reshape(-1, 1)

In [ ]:
f_dates = np.concatenate([np.repeat(pd.to_datetime('2019-7-28'), 1550, axis=0), np.repeat(
    pd.to_datetime('2019-8-28'), 1550, axis=0), np.repeat(pd.to_datetime('2019-9-28'), 1550, axis=0)], axis=0)
f_site = df_clean[-4650:]['site_code'].values.tolist()
f_product = df_clean[-4650:]['product_code'].values.tolist()

In [ ]:
submit = pd.DataFrame({'date': f_dates.tolist(), 'site_code': f_site, 'product_code': f_product, 'prediction':jul_sep.reshape(1, -1)[0]})

In [ ]:
def create_id(data: pd.DataFrame):
    id = []

    for i in range(len(data)):
        id_ = str(data['date'][i].year) + ' X ' + \
            str(data['date'][i].month) + ' X ' + \
            data['site_code'][i] + ' X ' + data['product_code'][i]
        id.append(id_)

    return id


In [ ]:
submit['ID'] = create_id(submit)

In [ ]:
submit[['ID', 'prediction']].to_csv('./submit.csv', index=False)